In [26]:
import sqlite3

dbs = ["db/std.db", "db/legal.db", "db/person.db"]

for db in dbs:
    connection = sqlite3.connect(db)
    cursor = connection.cursor()

    # Drop existing tables if they exist
    drop_ddl = [
        "DROP TABLE IF EXISTS \"association\";",
        "DROP TABLE IF EXISTS \"directed_relationship\";",
        "DROP TABLE IF EXISTS \"relationship\";",
        "DROP TABLE IF EXISTS \"primitive\";",
        "DROP TABLE IF EXISTS \"interface\";",
        "DROP TABLE IF EXISTS \"namespace\";",
        "DROP TABLE IF EXISTS \"internal_enumeration\";",
        "DROP TABLE IF EXISTS \"interface_uuid\";",
        "DROP TABLE IF EXISTS \"dependency\";"
    ]

    # Execute drop table commands
    for statement in drop_ddl:
        cursor.execute(statement)

    # SQL commands for creating new tables
    create_ddl = [
        """
        CREATE TABLE IF NOT EXISTS "namespace"(
            "uuid" TEXT PRIMARY KEY,
            "name" TEXT,
            "parent" INTEGER,
            FOREIGN KEY("parent") REFERENCES "namespace"("uuid")
        ) WITHOUT ROWID;
        """,
        """
        CREATE TABLE IF NOT EXISTS "interface"(
            "uuid" TEXT PRIMARY KEY,
            "namespace" INTEGER NOT NULL,
            "name" TEXT,
            FOREIGN KEY("namespace") REFERENCES "namespace"("uuid")
        ) WITHOUT ROWID;
        """,
        """
        CREATE TABLE IF NOT EXISTS "primitive"(
            "uuid" TEXT PRIMARY KEY,
            FOREIGN KEY("uuid") REFERENCES "interface"("uuid")
        ) WITHOUT ROWID;
        """,
        """
        CREATE TABLE IF NOT EXISTS "relationship"(
            "uuid" TEXT PRIMARY KEY
        ) WITHOUT ROWID;
        """,
        """
        CREATE TABLE IF NOT EXISTS "directed_relationship"(
            "uuid" TEXT PRIMARY KEY,
            "source" TEXT NOT NULL,
            "target" TEXT NOT NULL,
            FOREIGN KEY("uuid") REFERENCES "relationship"("uuid"),
            FOREIGN KEY("source") REFERENCES "interface"("uuid"),
            FOREIGN KEY("target") REFERENCES "interface"("uuid")
        ) WITHOUT ROWID;
        """,
        """
        CREATE TABLE IF NOT EXISTS "association"(
            "uuid" TEXT PRIMARY KEY,
            "name" TEXT,
            "domain" TEXT NOT NULL,
            "range" TEXT NOT NULL,
            "required" INTEGER DEFAULT 'FALSE',
            "multi" INTEGER DEFAULT 'FALSE',
            "exclusive" INTEGER DEFAULT 'FALSE',
            FOREIGN KEY("uuid") REFERENCES "relationship"("uuid"),
            FOREIGN KEY("domain") REFERENCES "interface"("uuid"),
            FOREIGN KEY("range") REFERENCES "interface"("uuid")
        ) WITHOUT ROWID;
        """
    ]

    # Execute create table commands
    for statement in create_ddl:
        cursor.execute(statement)

    # Commit changes and close the connection
    connection.commit()
    connection.close()

In [27]:
import sqlite3
import uuid

# std
connection = sqlite3.connect("db/std.db")
cursor = connection.cursor()

std_namespace_uuid = str(uuid.uuid4())
cursor.execute("""INSERT INTO "namespace" ("uuid", "name") VALUES (?, ?);""", (std_namespace_uuid, "std"))

primitives = ['Integer32', 'Boolean', 'String', 'Float64']
primitive_uuids = []

for primitive in primitives:
    primitive_uuid = str(uuid.uuid4())
    primitive_uuids.append(primitive_uuid)
    cursor.execute("""INSERT INTO "interface" ("uuid", "namespace", "name") VALUES (?, ?, ?);""", (primitive_uuid, std_namespace_uuid, primitive))
    cursor.execute("""INSERT INTO "primitive" ("uuid") VALUES (?);""", (primitive_uuid,))

primitives = dict(zip(primitives, primitive_uuids))

connection.commit()
connection.close()

#legal
connection = sqlite3.connect("db/legal.db")
cursor = connection.cursor()

legal_namespace_uuid = str(uuid.uuid4())
cursor.execute("""INSERT INTO "namespace" ("uuid", "name") VALUES (?, ?);""", (legal_namespace_uuid, "legal"))

legal_subject_interface_uuid = str(uuid.uuid4())
cursor.execute("""INSERT INTO "interface" ("uuid", "namespace", "name") VALUES (?, ?, 'LegalSubject');""", (legal_subject_interface_uuid, legal_namespace_uuid,))

connection.commit()
connection.close()

# person
connection = sqlite3.connect("db/person.db")
cursor = connection.cursor()

person_namespace_uuid = str(uuid.uuid4())
cursor.execute("""INSERT INTO "namespace" ("uuid", "name") VALUES (?, ?);""", (person_namespace_uuid, "person"))

person_interface_uuid = str(uuid.uuid4())
cursor.execute("""INSERT INTO "interface" ("uuid", "namespace", "name") VALUES (?, ?, 'Person');""", (person_interface_uuid, person_namespace_uuid,))

extension_relationship_uuid = str(uuid.uuid4())
cursor.execute("""INSERT INTO "relationship" ("uuid") VALUES (?);""", (extension_relationship_uuid,))
cursor.execute(
    """INSERT INTO "directed_relationship" ("uuid", "source", "target") VALUES (?, ?, ?);""",
    (extension_relationship_uuid, person_interface_uuid, legal_subject_interface_uuid)
)

age_association_uuid = str(uuid.uuid4())
cursor.execute("""INSERT INTO "relationship" ("uuid") VALUES (?);""", (age_association_uuid,))
cursor.execute(
    """INSERT INTO "association" ("uuid", "name", "domain", "range", "required", "multi", "exclusive") VALUES (?, ?, ?, ?, 'FALSE', 'FALSE', 'FALSE');""",
    (age_association_uuid, "age", person_interface_uuid, primitives["Integer32"])
)

connection.commit()
connection.close()


In [28]:
import sqlite3

connection = sqlite3.connect("db/merged.db")
cursor = connection.cursor()

drop_ddl = [
    "DROP TABLE IF EXISTS \"association\";",
    "DROP TABLE IF EXISTS \"directed_relationship\";",
    "DROP TABLE IF EXISTS \"relationship\";",
    "DROP TABLE IF EXISTS \"primitive\";",
    "DROP TABLE IF EXISTS \"interface\";",
    "DROP TABLE IF EXISTS \"namespace\";",
    "DROP TABLE IF EXISTS \"internal_enumeration\";",
    "DROP TABLE IF EXISTS \"interface_uuid\";",
    "DROP TABLE IF EXISTS \"dependency\";"
]

for statement in drop_ddl:
    cursor.execute(statement)

create_ddl = [
    """
    CREATE TABLE IF NOT EXISTS "namespace"(
        "uuid" TEXT PRIMARY KEY,
        "name" TEXT,
        "parent" INTEGER,
        FOREIGN KEY("parent") REFERENCES "namespace"("uuid")
    ) WITHOUT ROWID;
    """,
    """
    CREATE TABLE IF NOT EXISTS "interface"(
        "uuid" TEXT PRIMARY KEY,
        "namespace" INTEGER NOT NULL,
        "name" TEXT,
        FOREIGN KEY("namespace") REFERENCES "namespace"("uuid")
    ) WITHOUT ROWID;
    """,
    """
    CREATE TABLE IF NOT EXISTS "primitive"(
        "uuid" TEXT PRIMARY KEY,
        FOREIGN KEY("uuid") REFERENCES "interface"("uuid")
    ) WITHOUT ROWID;
    """,
    """
    CREATE TABLE IF NOT EXISTS "relationship"(
        "uuid" TEXT PRIMARY KEY
    ) WITHOUT ROWID;
    """,
    """
    CREATE TABLE IF NOT EXISTS "directed_relationship"(
        "uuid" TEXT PRIMARY KEY,
        "source" TEXT NOT NULL,
        "target" TEXT NOT NULL,
        FOREIGN KEY("uuid") REFERENCES "relationship"("uuid"),
        FOREIGN KEY("source") REFERENCES "interface"("uuid"),
        FOREIGN KEY("target") REFERENCES "interface"("uuid")
    ) WITHOUT ROWID;
    """,
    """
    CREATE TABLE IF NOT EXISTS "association"(
        "uuid" TEXT PRIMARY KEY,
        "name" TEXT,
        "domain" TEXT NOT NULL,
        "range" TEXT NOT NULL,
        "required" INTEGER DEFAULT 'FALSE',
        "multi" INTEGER DEFAULT 'FALSE',
        "exclusive" INTEGER DEFAULT 'FALSE',
        FOREIGN KEY("uuid") REFERENCES "relationship"("uuid"),
        FOREIGN KEY("domain") REFERENCES "interface"("uuid"),
        FOREIGN KEY("range") REFERENCES "interface"("uuid")
    ) WITHOUT ROWID;
    """
]

for statement in create_ddl:
    cursor.execute(statement)

connection.commit()

dbs = ["std", "legal", "person"]
tables = ["namespace", "interface", "primitive", "relationship", "directed_relationship", "association"]

for db in dbs:
    cursor.execute(f"ATTACH DATABASE 'db/{db}.db' AS {db}")

for db in dbs:
    for table in tables:
        cursor.execute(f"INSERT into {table} SELECT * FROM {db}.{table}")

connection.commit()

for db in dbs:
    cursor.execute(f"DETACH DATABASE '{db}'")

connection.close()

In [32]:
import pandas as pd
import sqlite3

connection = sqlite3.connect("db/merged.db")
cursor = connection.cursor()

query = """
    SELECT *
    FROM interface i
    JOIN namespace n ON i.namespace = n.uuid;
"""

resultset_df = pd.read_sql(query, connection)

connection.close()

resultset_df

,uuid,namespace,name,uuid,name,parent
0,300c3288-5a02-437f-b392-fbe1ab866a9b,741071fb-a17c-48fa-a76c-e194c2b229cf,String,741071fb-a17c-48fa-a76c-e194c2b229cf,std,None
1,4ea85861-6330-4893-8e66-a3ba341b4eb0,3093b28f-06f1-4e4a-afc4-b9a599bf3399,Person,3093b28f-06f1-4e4a-afc4-b9a599bf3399,person,None
2,8ded504f-96b0-4812-aaa2-fa8ec75d92c4,0b703a22-fcb6-4baf-9efa-b4e51752b467,LegalSubject,0b703a22-fcb6-4baf-9efa-b4e51752b467,legal,None
3,98214f36-baed-42a0-b214-6b84557ddaa5,741071fb-a17c-48fa-a76c-e194c2b229cf,Boolean,741071fb-a17c-48fa-a76c-e194c2b229cf,std,None
4,cc22db76-7e30-4204-b538-5ae7009f75df,741071fb-a17c-48fa-a76c-e194c2b229cf,Integer32,741071fb-a17c-48fa-a76c-e194c2b229cf,std,None
5,e78fe301-8b4c-41d0-9b4a-21636b38e676,741071fb-a17c-48fa-a76c-e194c2b229cf,Float64,741071fb-a17c-48fa-a76c-e194c2b229cf,std,None


In [35]:
import pandas as pd
import sqlite3

connection = sqlite3.connect("db/merged.db")
cursor = connection.cursor()

# List all properties of an interface.
query = """
    SELECT 
        id.name AS interface,
        a.name AS association, 
        ir.name AS range,
        a.required, 
        a.multi, 
        a.exclusive
    FROM association a
    JOIN interface id ON a.domain = id.uuid
    JOIN interface ir ON a.range = ir.uuid
    WHERE id.name = 'Person';
"""

resultset_df = pd.read_sql(query, connection)
 
connection.close()

resultset_df

,interface,association,range,required,multi,exclusive
0,Person,age,Integer32,FALSE,FALSE,FALSE
